# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,272.24,93,0,0.00,CA,1697361569
1,1,pimentel,-6.8367,-79.9342,292.31,88,100,5.14,PE,1697361570
2,2,yinchuan,38.4681,106.2731,293.75,15,0,1.00,CN,1697361354
3,3,anadyr,64.7500,177.4833,273.43,93,75,4.00,RU,1697361573
4,4,pyay,18.8167,95.2167,304.32,65,31,1.80,MM,1697361574


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 600,
    size = "humidity",
    scale = 1,
    color = "City",
    
)

# Display the map plot
humidity_map


:Points   [Lng,Lat]   (City)

In [4]:
import numpy
numpy.version.version

'1.23.5'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_conditions_df = city_data_df[(city_data_df["Max Temp"] < 280) &
                                   (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
Ideal_conditions_df.dropna(inplace=True)

# Display sample data
Ideal_conditions_df.head()

C:\Users\mac\AppData\Local\Temp\ipykernel_10152\201135026.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ideal_conditions_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,272.24,93,0,0.00,CA,1697361569
115,115,oroqen zizhiqi,50.5667,123.7167,278.25,34,0,5.97,CN,1697361709
220,220,tralee,52.2704,-9.7026,278.29,93,0,0.00,IE,1697361833
254,254,stanley,54.8680,-1.6985,279.70,74,0,4.12,GB,1697361872
286,286,idaho falls,43.4666,-112.0341,278.36,87,0,2.57,US,1697361907


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_conditions_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,thompson,CA,55.7435,-97.8558,93,
115,oroqen zizhiqi,CN,50.5667,123.7167,34,
220,tralee,IE,52.2704,-9.7026,93,
254,stanley,GB,54.8680,-1.6985,74,
286,idaho falls,US,43.4666,-112.0341,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel",  
    "limit": 1,  
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f"circle:{radius}@{lng},{lat}"
    #params["bias"] = f"circle:{radius}@{lng},{lat}"
    
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
thompson - nearest hotel: Thompson Inn
oroqen zizhiqi - nearest hotel: 大鮮卑酒店
tralee - nearest hotel: The Ashe Hotel
stanley - nearest hotel: Hotel 52
idaho falls - nearest hotel: Candlewood Suites
flin flon - nearest hotel: Copperbelt Hotel
perryton - nearest hotel: Ambassador Inn
kenai - nearest hotel: Beluga Lookout Hotel
yakeshi - nearest hotel: Baoling Hotel
la ronge - nearest hotel: La Ronge Hotel and Suites
pico truncado - nearest hotel: No hotel found
grande prairie - nearest hotel: Sandman Hotel Grande Prairie
saint cloud - nearest hotel: Courtyard


,City,Country,Lat,Lng,Humidity,Hotel Name
0,thompson,CA,55.7435,-97.8558,93,Thompson Inn
115,oroqen zizhiqi,CN,50.5667,123.7167,34,大鮮卑酒店
220,tralee,IE,52.2704,-9.7026,93,The Ashe Hotel
254,stanley,GB,54.8680,-1.6985,74,Hotel 52
286,idaho falls,US,43.4666,-112.0341,87,Candlewood Suites
363,flin flon,CA,54.7682,-101.8650,89,Copperbelt Hotel
414,perryton,US,36.4000,-100.8027,86,Ambassador Inn
430,kenai,US,60.5544,-151.2583,100,Beluga Lookout Hotel
454,yakeshi,CN,49.2833,120.7333,25,Baoling Hotel
492,la ronge,CA,55.1001,-105.2842,81,La Ronge Hotel and Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
points_with_hotel_info = hotel_df.hvplot.points(
    x='Lng', y='Lat',
    c='Humidity', size='Humidity',
    cmap='coolwarm',
    title='City Humidity',
    hover_cols=['City', 'Country', 'Hotel Name']
)


# Display the map
points_with_hotel_info.opts(
    tools=['hover'], width=800, height=600, colorbar=True,
    clim=(0, 100), line_color='black', line_width=1
)
points_with_hotel_info

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)